# "Investor In Town" -- Market/Property Evaluation Tool for Residential Real Estate Investors

Often times, real eastate investors may have only a day or two in a city to look at potential investment properties and even less time to research them.  Our project is designed to provide residential real estate investors with a tool that uses calculated metrics to help narrow their search for investment properties based on a set of criteria. These metrics can be both market (city/state) and property based so that investors can start by narrowing from a list of cities, then filter individual properties within that market to determine which rental houses show attractive investment ratios and ultimately, plot their stops on a map.

In this project, we will take a closer look at one of the most popular areas of the United States for residential real estate investors. The United States "Sun Belt" (which includes East Texas and the Southeastern US) has been offering attractive returns to investors over the last several years due to its high population and job growth.  But which cities in the Sun Belt offer the best investment properties, and which properties within those markets should an investor take a closer look at.  "Investor In Town" will answer these questions.

The following 10 cities will be evaluated in this model:

- Dallas, TX
- Austin, TX
- San Antonio, TX
- Houston, TX
- Atlanta, GA
- Charlotte, NC
- Jacksonville, FL
- Tampa, FL
- Orlando, FL
- Miami, FL


## Core Metrics

### We will use a set of core metrics to evaluate and filter data on a market and property level. A list of these metrics can be found below, along with their calculation and goal outcome.

**Market Level**

1. Annual Population Growth by state % (2010 - 2023)

2. Annual Job Growth by state % (2010 - 2023)

3. City Price to Rent Ratio: Median Home Price / Annual Median Rent (monthly rent * 12) 
Goal <= 15

4. Quality of Life Index

5. Cost of Living Index (measure of overall affordability)

6. Housing Price to Income Ratio (measure of housing affordability)


**Property Level**

1. Rent Ratio: Monthly Rent Estimate / Total Cost

Goal >.55%

2. Cap Rate:  Net Operating Income (NOI) / Total Cost

Traditional Rental:  *Goal >= 5%*
AirBNB Rental:  *Goal >= 9.5%*  

## Equations and Variables

# Formulas

1.  Total Cost = List Price + Closing Fees
2.  NOI = Gross Income - Annual Operating Expenses (OpEx)

### Based on the above calculations, we will need to gather the following individual data points (variable and potential source listed below):

1.  Address (Mashvisor API)
2.  City (Mashvisor API)
3.  State (Mashvisor API) 
4.  Zip Code (Mashvisor API)
5.  Latitude (Mashvisor API)
6.  Longitude (Mashvisor API)
7.  List Price (Mashvisor API)
8.  Square Feet (Mashvisor API)
9.  Bed/Bath (Mashvisor API)
10. Estimated Monthly Rent -- Traditional Rental (Mashvisor API)
11. Estimated Monthly Rent -- AirBNB Rental
12. Estimated Net Operating Income (NOI) -- Traditional Rental (Mashvisor API)
13. Estimated Net Operating Income (NOI) -- AirBNB Rental (Mashvisor API)
14. Median Home Price in a given city (Zillow csv: "Zillow Home Value Index")
15. Annual Median Rent in a given city (Zillow csv:  "Zillow Observed Rent Index")
16. Quality of Life Index (Numbeo)


In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import hvplot.pandas
import ipywidgets as widgets

In [ ]:
load_dotenv()
api_key = os.getenv("X_RAPID_API_KEY")

In [ ]:
# Create Population Growth By City Dictionary (Data from US Census)

pop_growth_dict = {'city': ['Austin, TX', 'Orlando, FL', 'Houston, TX', 'San Antonio, TX', 'Dallas, TX', 'Charlotte, NC', 'Jacksonville, FL', 'Tampa, FL', 'Atlanta, GA', 'Miami, FL'],
             'pop_growth_by_city': [25.84, 19.69, 17.36, 16.59, 16.58, 15.33, 13.71, 13.07, 12.15, 11.11]}

pop_growth_df = pd.DataFrame.from_dict(pop_growth_dict)
pop_growth_df.set_index("city")

In [ ]:
# Plot Population Growth by City

pop_growth_df.hvplot.bar(
    x='city', 
    y='pop_growth_by_city',
    ylabel='Population Growth (%)', 
    xlabel='City',
    rot=45,
    ylim=(5, 45),
    title='Population Growth by City -- Major "Sun Belt" Markets', 
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="orange")

In [ ]:
# Create Job Growth By City Dictionary (Data from US Census)

job_growth_dict = {'city': ['Dallas, TX', 'Charlotte, NC', 'Atlanta, GA', 'Miami, FL', 'Tampa, FL', 'Orlando, FL', 'Jacksonville, FL', 'Austin, TX', 'San Antonio, TX', 'Houston, TX'],
             'job_growth_by_city': [6.5, 5.7, 5.4, 5.2, 5.1, 4.9, 4.8, 4.7, 4.6, 3.2]}

job_growth_df = pd.DataFrame.from_dict(job_growth_dict)
job_growth_df.set_index("city")

In [ ]:
# Plot Job Growth by City

job_growth_df.hvplot.bar(
    x='city', 
    y='job_growth_by_city',
    ylabel='Job Growth (%)', 
    xlabel='City',
    rot=45,
    ylim=(2, 8),
    title='Job Growth by City -- Major "Sun Belt" Markets', 
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="orange")

In [ ]:
# Begin City-Level Data 
# Create Quality of Life Index (QOLI) Dictionary (Data from Numbeo)

qoli_dict = {'city': ['Jacksonville, FL', 'Orlando, FL', 'Tampa, FL', 'San Antonio, TX', 'Austin, TX', 'Dallas, TX', 'Charlotte, NC', 'Houston, TX', 'Atlanta, GA', 'Miami, FL'],
             'quality_of_life_index': [176.7, 176.6, 176.6, 176.5, 176.4, 176.2, 174.4, 168.5, 166.8, 153.6]}

qoli_df = pd.DataFrame.from_dict(qoli_dict)
qoli_df.set_index("city")

In [ ]:
# Plot Quality of Life Index by City

qoli_df.hvplot.bar(
    x='city', 
    y='quality_of_life_index',
    ylabel='Quality of Life Index', 
    xlabel='City',
    rot=45,
    ylim=(125, 185),
    title='Quality of Life Index -- Major "Sun Belt" Markets', 
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="orange")

In [ ]:
# Create Cost of Living Index (COLI) Dictionary (Data from Numbeo)

coli_dict = {'city': ['San Antonio, TX', 'Jacksonville, FL', 'Houston, TX', 'Orlando, FL', 'Austin, TX', 'Atlanta, GA', 'Tampa, FL', 'Dallas, TX', 'Miami, FL', 'Charlotte, NC'],
             'cost_of_living_index': [67.4, 67.9, 70.0, 72.1, 74.0, 74.0, 75.2, 77.4, 81.5, 81.6]}

coli_df = pd.DataFrame.from_dict(coli_dict)
coli_df.set_index("city")
coli_df

In [ ]:
# Plot Cost of Living Index by City

coli_df.hvplot.bar(
    x='city', 
    y='cost_of_living_index',
    ylabel='Cost of Living Index', 
    xlabel='City',
    rot=45,
    ylim=(50, 100),
    title='Cost of Living Index -- Major "Sun Belt" Markets', 
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="orange")

In [ ]:
# Create Housing Price to Income Ratio (p2i) Dictionary (Data from Numbeo)

p2i_dict = {'city': ['Houston, TX', 'Dallas, TX', 'Atlanta, GA', 'Charlotte, NC', 'San Antonio, TX', 'Jacksonville, FL', 'Orlando, FL', 'Tampa, FL', 'Miami, FL', 'Austin, TX'],
             'price_to_income_ratio': [2.2, 2.8, 3.1, 3.7, 4.2, 4.3, 4.5, 4.5, 5.3, 6.1]}

p2i_df = pd.DataFrame.from_dict(p2i_dict)
p2i_df.set_index("city")
p2i_df

In [ ]:
# Plot Housing Price to Income Ratio by City (Data from Numbeo)

p2i_df.hvplot.bar(
    x='city', 
    y='price_to_income_ratio',
    ylabel='Price to Income Ratio', 
    xlabel='City',
    rot=45,
    ylim=(0, 8),
    title='Housing Price to Income Ratio -- Major "Sun Belt" Markets', 
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="orange")

In [ ]:
state = widgets.Dropdown(
    options=[('Florida', 'FL'), ('Texas', 'TX'), ('North Carolina', 'NC'), ('Georgia', 'GA')],
    value='FL',
    description='State:',
)

state

In [ ]:



texas = ['Houston', 'Dallas', 'San Antonio', 'Austin']
florida = ['Orlando', 'Tampa', 'Miami', 'Jacksonville']
georgia = ['Atlanta']
north_carolina = ['Charlotte']



def state_selection(selected):
    options = []
    if selected == 'FL':
        for option in florida:
            options.append(option)
        return options
    elif selected == 'TX':
        for option in texas:
            options.append(option)
        return options
    elif selected == 'GA':
        options = georgia
        return options
    else:
        options = north_carolina
        display(options)
        return options 

selected_state = state_selection(state.value)

city = widgets.Dropdown(
    options=selected_state,
    value=None,
    description='City:',
)

city

In [ ]:
# Call mashvisor api for property dataframe
url = f"https://api.mashvisor.com/v1.1/client/city/properties/{state.value}/{city.value}"
display(url)
payload={}
headers = {
  'x-api-key': '6c4620ed-a730-48fb-9b1d-8d93ea627343'
}
response = requests.request("GET", url, headers=headers, data=payload)

#reformat json data to dictionary
data = response.json()
mashvisor_data = json.dumps(data, indent=4)
new_dict = json.loads(mashvisor_data)
new_dict
mashvisor_dict = new_dict['content']['properties']
#mashvisor_dict

In [ ]:
# Create new property dictionary and convert to a pandas dataframe
new_list = []
for prop in mashvisor_dict:
    new_dict = {
        "property_id": prop["id"],
        "address": prop["address"],
        "city": prop["city"],
        "state": prop["state"],
        "zip_code": prop["zip_code"],
        "latitude": prop["latitude"],
        "longitude": prop["longitude"],
        "list_price": prop["list_price"],
        "square_ft": prop["sqft"],
        "beds": prop["beds"],
        "baths": prop["baths"],
        "price_per_sqft": prop["price_per_sqft"],
        "trad_monthly_rent": prop["traditional_rental"],
        "airbnb_monthly_rent": prop["airbnb_rental"],
        #"reg_ROI": prop["traditional_ROI"],
        #"airbnb_ROI": prop["airbnb_ROI"],
        "trad_cap_rate": prop["traditional_cap"],
        "airbnb_cap_rate": prop["airbnb_cap"],
        #"reg_COC": ["COC"][1],
        #"airbnb_COC": ["COC"][0]
    }        
    new_list.append(new_dict)
new_list
property_df = pd.DataFrame(new_list)
property_df.dropna()
property_df.head()

In [ ]:
# Add columns, clean and sort data, and create a property dataframe to find the top 20 properties on the market
property_df["rent_ratio"] = property_df["trad_monthly_rent"] / property_df["list_price"]
property_df["rent_ratio"] = property_df["rent_ratio"] * 100
property_df.set_index("property_id")
property_df.sort_values("trad_cap_rate", ascending=False)
property_df.head()

In [ ]:
# Create Market Summary Dataframe
market_summary_df = pd.DataFrame({'avg_list_price': property_df["list_price"].mean(),
                          'avg_sqft': property_df["square_ft"].mean(),
                          'avg_monthly_rent_trad': property_df["trad_monthly_rent"].mean(),
                          'avg_monthly_rent_airbnb': property_df["airbnb_monthly_rent"].mean(),
                          'avg_cap_rate_trad': property_df["trad_cap_rate"].mean(),
                          'avg_cap_rate_airbnb': property_df["airbnb_cap_rate"].mean(),
                          'avg_rent_ratio': avg_monthly_rent_trad / avg_list_price
                         }, index=[0])

market_summary_df

In [ ]:
# Futher filter data to return only those homes that meet the goal criteria listed in the project description 
goal_df = property_df.loc[(property_df['trad_cap_rate'] >= 5) & (property_df['rent_ratio'] > .55) & (property_df['airbnb_cap_rate'] > 9.5)]
goal_df.sort_values("trad_cap_rate", ascending=False)
goal_df.head()

In [ ]:
# Map the properties that meet the goal criteria

goal_property_plot = goal_df.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True, 
    color='address',
    alpha=0.8,
    size = 500,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    )

# Show the Plot
goal_property_plot